# Entrenamiento LoRA (H100) con dataset estructurado

Este notebook prepara dataset estructurado con schema JSON y entrena un LoRA de alta calidad en H100.


In [ ]:
!pip install -q --upgrade "transformers>=4.48.0" "trl==0.11.4" "peft>=0.12.0" "datasets>=2.20.0" "accelerate>=0.31.0" "bitsandbytes>=0.43.0"


## 1) Clonar repo


In [ ]:
!git clone https://github.com/TU_USUARIO/TU_REPO.git
%cd TU_REPO/jupyter-notebook-backend/python/ia-jupyterNotebook


## 2) Preparar dataset HF (streaming)


In [ ]:
!python prepare_hf_dataset.py \
  --dataset jupyter-agent/jupyter-agent-dataset \
  --split non_thinking \
  --streaming \
  --output hf_jupyter_messages.jsonl \
  --max-samples 40000 \
  --include-packages ""

!wc -l hf_jupyter_messages.jsonl


## 3) Convertir a dataset estructurado (schema JSON final)


In [ ]:
!python prepare_structured_dataset.py \
  --input hf_jupyter_messages.jsonl \
  --output hf_jupyter_structured.jsonl \
  --max-samples 40000

!wc -l hf_jupyter_structured.jsonl


## 4) Entrenar LoRA (H100 - alta calidad)


In [ ]:
!PYTORCH_ALLOC_CONF=expandable_segments:True python train_lora_qwen.py \
  --model-name Qwen/Qwen3-4B-Instruct-2507 \
  --train-file hf_jupyter_structured.jsonl \
  --output-dir qwen3-jupyter-lora \
  --max-steps 2000 \
  --max-seq-length 1536 \
  --per-device-train-batch-size 1 \
  --gradient-accumulation-steps 8 \
  --lora-r 32 \
  --lora-alpha 64


## 5) Empaquetar y descargar adapter


In [ ]:
!zip -r qwen3-jupyter-lora.zip qwen3-jupyter-lora
from google.colab import files
files.download('qwen3-jupyter-lora.zip')
